<a href="https://colab.research.google.com/github/izzul112/ML_MLO-Submission_1/blob/main/mlop_submission1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Submission 1: Klasifikasi Email spam atau bukan spam**

Nama: Habib Azizul Haq

Username dicoding: kuroba_izzul

Mendapat sebuah email dari seorang kenalan, membacanya dan kemudian membalasnya adalah hal yang cukup menyenangkan. Namun terkadang kita juga mendapat email yang berisi sesuatu yang tidak kita pahami atau mengerti, nah itulah yang sering kita sebut dengan email spam. Tentunya kita semua tidak ingin email spam itu muncul di beranda email kita, karena selain mengganggu dan tak sedap di pandang mata. Email spam juga terkadang berisi link-link yang berbahaya. Tentunya kita tidak ingin mengalami salah membuka email, terus juga secara tidak sengaja menekan link email tersebut, bisa-bisa semua data pribadi kita diambil atau kita secara tidak sadar telah memberikan akses kepada orang yang tidak bertanggung jawab kepada gawai kita.

Tenang tidak perlu panik kita akan membuat sebuah model machine learning untuk menjawab permasalahan tersebut. Kita akan membuat model untuk medeteksi apakah sebuah email itu spam atau bukan.

## **Tahap Persiapan**

Di tahap ini kita akan menginsatall semua library yang di butuhkan serta membuat virtual environment.

1. Install condacolab

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


2. Cek versinya jika perlu

In [ ]:
!conda --version

conda 22.9.0


3. Buat conda virtual environment, kita berinama mlops-tfx-sub-1

In [ ]:
!conda create --name mlops-tfx-sub-1 python==3.9.15

Solving environment: \ failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.1.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/mlops-tfx-sub-1

  added / updated specs:
    - python==3.9.15


The following NEW packages will be INSTALLED:

  _libgcc_mutex      conda-forge/linux-64::_libgcc_mutex-0.1-conda_forge None
  _openmp_mutex      conda-forge/linux-64::_openmp_mutex-4.5-2_gnu None
  bzip2              conda-forge/linux-64::bzip2-1.0.8-h7f98852_4 None
  ca-certificates    conda-forge/linux-64::ca-certificates-2022.12.7-ha878542_0 None
  ld_impl_linux-64   conda-forge/linux-64::ld_impl_li

4. Aktifkan conda virtual environment-nya

In [ ]:
!conda activate mlops-tfx-sub-1


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




In [ ]:
!git clone https://github.com/izzul112/ML_MLO-Submission_1.git

fatal: destination path 'ML_MLO-Submission_1' already exists and is not an empty directory.


In [ ]:
!pip install jupyter scikit-learn tensorflow tfx==1.11.0 flask joblib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install -r /content/ML_MLO-Submission_1/requirements.txt --user

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached absl_py-1.1.0-py3-none-any.whl (123 kB)
  Using cached apache_beam-2.39.0-cp38-cp38-manylinux2010_x86_64.whl (12.3 MB)
  Using cached attrs-20.3.0-py2.py3-none-any.whl (49 kB)
  Using cached beautifulsoup4-4.11.1-py3-none-any.whl (128 kB)
  Using cached bleach-5.0.1-py3-none-any.whl (160 kB)
  Using cached charset_normalizer-2.1.0-py3-none-any.whl (39 kB)
  Using cached cloudpickle-2.1.0-py3-none-any.whl (25 kB)
  Using cached colorama-0.4.5-py2.py3-none-any.whl (16 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached debugpy-1.6.2-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.8 MB)
  Using cached entrypoints-0.4-py3-none-any.whl (5.3 kB)
  Using cached fastavro-1.5.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.6 MB)
  Using cached fasteners-0.17.3-py3-none-any.whl (18 kB)
  Using cach

In [ ]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

## **Set Variabel**

In [ ]:
PIPELINE_NAME = "emailspam-pipeline"
SCHEMA_PIPELINE_NAME = "emailspam-tfdv-schema"

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# from absl import logging
# logging.set_verbosity(logging.INFO)

In [ ]:
DATA_ROOT = "/content/ML_MLO-Submission_1/data"

## **Membuat Tahapan Data Ingestion**

**1. example_gen**

In [ ]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [ ]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

In [ ]:
interactive_context.run(example_gen)

RuntimeError: ignored

## **Membuat Tahapan Data Validation**

**Membuat summary statistic**

**2. StatisticGen**

In [ ]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
 
 
interactive_context.run(statistics_gen)

In [ ]:
interactive_context.show(statistics_gen.outputs["statistics"])

**Membuat data schema**

**3. SchemaGen**

In [ ]:
schema_gen = SchemaGen(    statistics=statistics_gen.outputs["statistics"])

interactive_context.run(schema_gen)

In [ ]:
interactive_context.show(schema_gen.outputs["schema"])

**Mengidentifikasi anomali pada dataset**

**4. ExampleValidator**

In [ ]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

Untuk menampilkan hasil dari validasi dari komponen tersebut, kita bisa menjalankan kode berikut.

In [ ]:
interactive_context.show(example_validator.outputs['anomalies'])

## **Membuat Tahapan Data Preprocessing**

In [ ]:
TRANSFORM_MODULE_FILE = "emailspam_transform.py"

In [ ]:
%%writefile {TRANSFORM_MODULE_FILE}
import tensorflow as tf
LABEL_KEY = "is_sarcastic"
FEATURE_KEY = "headline"
def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"
def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features
    
    Args:
        inputs: map from feature keys to raw features.
    
    Return:
        outputs: map from feature keys to transformed features.    
    """
    
    outputs = {}
    
    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(inputs[FEATURE_KEY])
    
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)
    
    return outputs

**5. Transform**

In [ ]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)